# Libraries

In [2]:
import numpy as np
import pandas as pd
from time import perf_counter
import matplotlib.pyplot as plt

# Excercise 1

In [6]:
def A1(float_type, n):
    A = np.array([[1 / (i + j - 1) if i != 1 else 1 
        for j in range(1, n + 1)] 
        for i in range(1, n + 1)]).astype(float_type)
    return A

def gauss(A, B):
    n = np.shape(A)[0]
    C = np.hstack([A, B.reshape((n, 1))]).astype(np.float64)

    for i in range(n):
        for j in range(i + 1, n):
            ratio = C[j][i] / C[i][i]
            C[j] = C[j] - ratio * C[i]

    X = C[:, n]
    X[n - 1] /= C[n - 1][n - 1]
    for i in range(n - 2, -1, -1):
        X[i] -= np.sum(C[i][i + 1:n] * X[i + 1:n])
        X[i] /= C[i][i]
    return X

def ex1():
    result = []
    ns = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 100, 150, 300]
    float_types = [np.float32, np.float64]

    for n in ns:
        for ft in float_types:
            A = A1(ft, n)
            X_vec = np.array([1 if i % 2 == 0 else -1 for i in range(n)]).astype(ft)
            B = np.matmul(A, X_vec)
            X = gauss(A, B)
            norm = format(np.linalg.norm(X_vec - X), '.3e')
            result += [norm]
    df = pd.DataFrame(data={"n": ns,
                            "float32": result[::2],
                            "float64": result[1::2]})
        
    return df

datas = ex1()
datas

,n,float32,float64
0,3,3.345e-06,0.000e+00
1,4,6.647e-15,3.019e-13
2,5,2.184e-03,9.229e-12
3,6,9.070e-02,3.638e-10
4,7,2.787e+00,1.361e-08
5,8,6.104e-03,1.203e-07
6,9,4.948e+01,5.401e-07
7,10,1.816e+01,1.662e-04
8,11,1.378e+01,1.223e-02
9,12,4.693e+00,1.214e+00


# Excercise 2

In [ ]:
def A2(float_type, n):
    A = np.zeros((n, n)).astype(float_type)
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]

    return A

def ex2():
    result = []
    ns = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 100, 150, 300]
    float_types = [np.float32, np.float64]

    for n in ns:
        for ft in float_types:
            A = A2(ft, n)
            X_vec = np.array([1 if i % 2 == 0 else -1 for i in range(n)]).astype(ft)
            B = np.matmul(A, X_vec)
            X = gauss(A, B)
            norm = format(np.linalg.norm(X_vec - X), '.3e')
            result += [norm]
    df = pd.DataFrame(data={"n": ns,
                            "float32": result[::2],
                            "float64": result[1::2]})
        
    return df

datas = ex2()
datas

,n,float32,float64
0,3,6.862e-08,3.140e-16
1,4,4.443e-08,2.483e-16
2,5,1.075e-07,4.154e-16
3,6,1.774e-07,9.742e-16
4,7,1.921e-07,1.695e-15
5,8,1.791e-07,4.672e-15
6,9,1.397e-06,3.310e-15
7,10,1.402e-06,3.083e-15
8,11,1.583e-06,4.421e-15
9,12,4.880e-06,1.980e-14


# Conditionality indicator

In [11]:
def A1_(n):
    return np.array([[1 / (i + j - 1) if i != 1 else 1 
                      for j in range(1, n + 1)] 
                      for i in range(1, n + 1)])

def A2_(n):
    A = np.zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]
    return A

def norm(A):
    n = len(A)
    return max(sum(A[i][j] for j in range(n)) for i in range(n))

def conditioning_factor(A):
    A_inv = np.linalg.inv(A)
    return norm(A_inv) * norm(A)

def condition_number():
    ns = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 100, 150, 300]
    result = []
    for n in ns:
        con_num_1 = format(conditioning_factor(A1_(n)), '.3e')
        con_num_2 = format(conditioning_factor(A2_(n)), '.3e')
        result += [con_num_1, con_num_2]
    df = pd.DataFrame(data={"n":ns,
                            "ex 1 condition number":result[::2],
                            "ex 2 condition number":result[1::2]})
    return df

condition_df = condition_number()
condition_df


,n,ex 1 condition number,ex 2 condition number
0,3,2.160e+02,1.444e+00
1,4,2.880e+03,1.833e+00
2,5,2.800e+04,2.233e+00
3,6,2.268e+05,2.644e+00
4,7,1.630e+06,3.032e+00
5,8,1.286e+07,3.448e+00
6,9,1.120e+08,3.849e+00
7,10,8.841e+08,4.249e+00
8,11,6.463e+09,4.659e+00
9,12,4.142e+10,5.055e+00


# Excercise 3

In [13]:
def A3(n):
    A = np.zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            if i == j:
                A[i - 1][j - 1] = -3 * i - 7
            elif j == i + 1:
                A[i - 1][j - 1] = i
            elif i > j == i - 1:
                A[i - 1][j - 1] = 3 / i

    return A

def thomas(A, B):
    n = np.shape(A)[0]
    C = np.zeros(n)
    C[0] = A[0][0]

    X = np.zeros(n)
    X[0] = B[0]

    for i in range(1, n):
        ratio = A[i][i - 1] / C[i - 1]
        C[i] = A[i][i] - ratio * A[i - 1][i]
        X[i] = B[i] - ratio * X[i - 1]

    X[n - 1] = X[n - 1] / C[n - 1]
    for i in range(n - 2, -1, -1):
        X[i] = (X[i] - A[i][i + 1] * X[i + 1]) / C[i]

    return X    

def ex3():
    result = []
    ns = [2, 3, 4, 5, 6, 7, 10, 15, 20, 50, 100, 150, 200, 300]

    for n in ns:
        A = A3(n)
        X_vec = np.array([1 if i % 2 == 0 else -1 for i in range(n)])
        B = np.matmul(A, X_vec)
        g_start = perf_counter()
        X_g = gauss(A, B)
        g_end = perf_counter()
        g_time = g_end - g_start
        err_g = format(np.linalg.norm(X_vec - X_g), '.2e')

        t_start = perf_counter()
        X_t = thomas(A, B)
        t_end = perf_counter()
        t_time = t_end - t_start
        err_t = format(np.linalg.norm(X_vec - X_t), '.2e')

        result += [err_g, err_t, g_time, t_time]

    df = pd.DataFrame(data={"n": ns,
                            "gauss": result[::4],
                            "thomas": result[1::4],
                            "gauss time [s]": result[2::4],
                            "thomas time [s]": result[3::4]})
    return df

datas = ex3()
datas

,n,gauss,thomas,gauss time [s],thomas time [s]
0,2,0.00e+00,0.00e+00,0.000190,0.000016
1,3,0.00e+00,0.00e+00,0.000282,0.000018
2,4,0.00e+00,0.00e+00,0.000158,0.000028
3,5,0.00e+00,0.00e+00,0.000280,0.000023
4,6,0.00e+00,0.00e+00,0.000365,0.000017
5,7,0.00e+00,0.00e+00,0.000242,0.000018
6,10,2.48e-16,2.48e-16,0.000587,0.000021
7,15,2.48e-16,2.48e-16,0.001468,0.000040
8,20,4.84e-16,4.84e-16,0.000818,0.000045
9,50,8.16e-16,8.16e-16,0.003631,0.000106
